# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

ans:在推导朴素贝叶斯公式时，计算P(a1,a2,a3|b)的概率,假设特征a1,a2,a3是相互独立的，则可以近似变为P(a1|b)*P(a2|b)*P(a3|b)

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

ans:

MAP(maximum a posterior):最大后验概率MAP=argmax P(h|D) = argmax P(D|h)P(h),即在给定的数据集中有很多拟合数据集D的线性函数h，选出最可能拟合的D的函数h的概率为MAP。

ML(maximum likelihood):ML=argmax P(D|h),找一个函数可以最大程度拟合数据集D的概率,假设找所有函数h的概率是相等的，不像MAP是有偏好的，所以可以可以把MAP中的P(h)去掉得到ML。

#### 3. What is support vector in SVM?

ans:support vector是在边界上的向量，对边界有贡献的向量。经过计算后，样本中alpha大于0的点。

#### 4. What is the intuition behind SVM ?

ans:

#### 5. Shortly describ what 'random' means in random forest ?

ans:在对一个多feature的分类任务使用随机森林时，1.随机的初始化样本，2.随机的选取teature进行建树

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

ans:计算所有叶子结点的权重，并用该权重计算分割点的损失函数，损失函数最小的点即为需要分割的点。

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).